In [1]:
def first_moves(input, num_moves):
    if input == []: return ""
    if input[0][0:3]!="W1.": return ""
    output_str=""
    i = 0
    for item in input:
        if(i>=num_moves): break
        output_str+=item[3:]
        i+=1
    return output_str

In [2]:
def openings(moves):
    for index, opening in openings_list.iterrows():
        len_opening = len(opening[0])

        if len(moves)<len_opening: 
            continue

        if moves[:len_opening]==opening[0]: return opening[1]
    return "Other"
            

In [3]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

dataUri = "gs://big-data-bucket-1/chess/all_with_filtered_anotations_since1998.txt"
#sc.stop()
sc = pyspark.SparkContext()
spark = SparkSession(sc)

openings_list = pd.read_csv("gs://big-data-bucket-1/chess/openings.csv")
openings_list

lines = sc.textFile(dataUri)
words = lines.map(lambda line:line.split())

games = words.filter(lambda line: (line[0]!="#") and (line[6]=='date_false') and (line[7]=='result_false') and (line[11]=='setup_false'))
games.take(5)

[['1',
  '2000.03.14',
  '1-0',
  '2851',
  'None',
  '67',
  'date_false',
  'result_false',
  'welo_false',
  'belo_true',
  'edate_true',
  'setup_false',
  'fen_false',
  'result2_false',
  'oyrange_false',
  'blen_false',
  '###',
  'W1.d4',
  'B1.d5',
  'W2.c4',
  'B2.e6',
  'W3.Nc3',
  'B3.Nf6',
  'W4.cxd5',
  'B4.exd5',
  'W5.Bg5',
  'B5.Be7',
  'W6.e3',
  'B6.Ne4',
  'W7.Bxe7',
  'B7.Nxc3',
  'W8.Bxd8',
  'B8.Nxd1',
  'W9.Bxc7',
  'B9.Nxb2',
  'W10.Rb1',
  'B10.Nc4',
  'W11.Bxc4',
  'B11.dxc4',
  'W12.Ne2',
  'B12.O-O',
  'W13.Nc3',
  'B13.b6',
  'W14.d5',
  'B14.Na6',
  'W15.Bd6',
  'B15.Rd8',
  'W16.Ba3',
  'B16.Bb7',
  'W17.e4',
  'B17.f6',
  'W18.Ke2',
  'B18.Nc7',
  'W19.Rhd1',
  'B19.Ba6',
  'W20.Ke3',
  'B20.Kf7',
  'W21.g4',
  'B21.g5',
  'W22.h4',
  'B22.h6',
  'W23.Rh1',
  'B23.Re8',
  'W24.f3',
  'B24.Bb7',
  'W25.hxg5',
  'B25.fxg5',
  'W26.d6',
  'B26.Nd5+',
  'W27.Nxd5',
  'B27.Bxd5',
  'W28.Rxh6',
  'B28.c3',
  'W29.d7',
  'B29.Re6',
  'W30.Rh7+',
  'B30.Kg8',
 

In [4]:
next = games.map(lambda line: (line[1],line[1][:4],line[1][:3]+"0",line[2],line[3],line[4],line[5],first_moves(line[17:],1),openings(first_moves(line[17:],6))))
next = next.filter(lambda line: (line[1]!="0206") and (line[6]!="0"))
next.take(5)

[('2000.03.14',
  '2000',
  '2000',
  '1-0',
  '2851',
  'None',
  '67',
  'd4',
  "Queen's Gambit Declined"),
 ('2000.03.14',
  '2000',
  '2000',
  '1-0',
  '2851',
  'None',
  '53',
  'e4',
  'Scandinavian Defense'),
 ('1999.11.20',
  '1999',
  '1990',
  '1-0',
  '2851',
  'None',
  '57',
  'e4',
  'Italian Game'),
 ('1999.11.20',
  '1999',
  '1990',
  '1-0',
  '2851',
  'None',
  '49',
  'e4',
  'Scandinavian Defense'),
 ('2000.02.20',
  '2000',
  '2000',
  '1/2-1/2',
  '2851',
  '2633',
  '97',
  'e4',
  'Rui Lopez')]

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import col

df = next.toDF()
df = df.withColumn("_5", df["_5"].cast(IntegerType()))
df = df.withColumn("_6", df["_6"].cast(IntegerType()))
df = df.withColumn("_7", df["_7"].cast(IntegerType()))
df = df.withColumnRenamed("_1","date").withColumnRenamed("_2","year").withColumnRenamed("_3","decade").withColumnRenamed("_4","result").withColumnRenamed("_5","welo").withColumnRenamed("_6","belo").withColumnRenamed("_7","moves").withColumnRenamed("_8","first_mv").withColumnRenamed("_9","opening")
df.show()

+----------+----+------+-------+----+----+-----+--------+--------------------+
|      date|year|decade| result|welo|belo|moves|first_mv|             opening|
+----------+----+------+-------+----+----+-----+--------+--------------------+
|2000.03.14|2000|  2000|    1-0|2851|null|   67|      d4|Queen's Gambit De...|
|2000.03.14|2000|  2000|    1-0|2851|null|   53|      e4|Scandinavian Defense|
|1999.11.20|1999|  1990|    1-0|2851|null|   57|      e4|        Italian Game|
|1999.11.20|1999|  1990|    1-0|2851|null|   49|      e4|Scandinavian Defense|
|2000.02.20|2000|  2000|1/2-1/2|2851|2633|   97|      e4|           Rui Lopez|
|2000.05.24|2000|  2000|1/2-1/2|2851|2748|   52|      d4|Queen's Pawn Open...|
|2000.06.19|2000|  2000|    1-0|2851|2191|   79|      e4|    Sicilian Defense|
|1999.11.20|1999|  1990|    1-0|2851|null|   71|      d4|         Indian Game|
|2000.06.19|2000|  2000|1/2-1/2|2851|2175|   72|      c4|     English Opening|
|2000.01.18|2000|  2000|    1-0|2851|2646|   49|    

In [6]:
from pyspark.sql.functions import count, avg, desc, asc

#df.groupBy("opening").count().orderBy(desc("count")).show()

In [7]:
df.cache().count

<bound method DataFrame.count of DataFrame[date: string, year: string, decade: string, result: string, welo: int, belo: int, moves: int, first_mv: string, opening: string]>

In [8]:
#df.write.format('csv').option('header',True).mode('overwrite').save('gs://big-data-bucket-1/chess/output.csv')

In [ ]:
df.toPandas().to_csv('gs://big-data-bucket-1/chess/output.csv')

In [ ]:
for x in range(15): print(x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
